In [26]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler

In [27]:
pd.set_option('display.max_columns', None)

data_df = pd.read_csv('movies.csv')

trimmed_votes_df = data_df.drop(data_df[data_df.votes <1000].index)

trimmed_df = trimmed_votes_df.dropna()

In [28]:
trimmed_df.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [29]:
# Splitting the released date column and extracting the date
def extract_date(date_str):
    date_only_str = date_str.split('(')[0].strip()  # Extracting date part
    date_obj = pd.to_datetime(date_only_str, format="%B %d, %Y", errors='coerce')
    return date_obj

# Apply the function to the 'released' column
trimmed_df['release_date'] = trimmed_df['released'].apply(extract_date)

trimmed_df['formatted_date'] = trimmed_df['release_date'].dt.strftime("%m-%d-%Y")
clean_df = trimmed_df.drop(columns=['release_date', 'released'])

# Display the DataFrame with only the date extracted
clean_df.head()

C:\Users\dasam\AppData\Local\Temp\ipykernel_10980\2600568437.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_df['release_date'] = trimmed_df['released'].apply(extract_date)
C:\Users\dasam\AppData\Local\Temp\ipykernel_10980\2600568437.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_df['formatted_date'] = trimmed_df['release_date'].dt.strftime("%m-%d-%Y")


,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime,formatted_date
0,The Shining,R,Drama,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,06-13-1980
1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,07-02-1980
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,06-20-1980
3,Airplane!,PG,Comedy,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,07-02-1980
4,Caddyshack,R,Comedy,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,07-25-1980


In [30]:
# Determining the success/failure of the movie
clean_df['factors']= (clean_df['gross'] / clean_df['budget'])
clean_df.head()

,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime,formatted_date,factors
0,The Shining,R,Drama,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,06-13-1980,2.473620
1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,07-02-1980,13.078468
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,06-20-1980,29.909726
3,Airplane!,PG,Comedy,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,07-02-1980,23.843868
4,Caddyshack,R,Comedy,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,07-25-1980,6.641057


In [31]:
# Creating a new column and categorizing it (success or failure) based on the condition specified
clean_df['success'] = clean_df['factors'].map(lambda x: x>3).astype(bool)
clean_df.head()

,name,rating,genre,year,score,votes,director,writer,star,country,budget,gross,company,runtime,formatted_date,factors,success
0,The Shining,R,Drama,1980,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,06-13-1980,2.473620,False
1,The Blue Lagoon,R,Adventure,1980,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,07-02-1980,13.078468,True
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,06-20-1980,29.909726,True
3,Airplane!,PG,Comedy,1980,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,07-02-1980,23.843868,True
4,Caddyshack,R,Comedy,1980,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,07-25-1980,6.641057,True


In [33]:
# KNN Time

# Split target column from dataset
y = clean_df['success']
X = clean_df.drop(columns='success')

In [40]:
# Insert dummies for categorical values
X = pd.get_dummies(X)

In [41]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [45]:
# X_train.shape
# X_test.shape

In [46]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=4)

In [65]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=4)

In [66]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array([False, False, False, ..., False, False, False])

In [67]:
# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[858, 388],
       [ 53,  47]], dtype=int64)

In [68]:
# Print classification report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.94      0.69      0.80      1246
        True       0.11      0.47      0.18       100

    accuracy                           0.67      1346
   macro avg       0.52      0.58      0.49      1346
weighted avg       0.88      0.67      0.75      1346

